In [1]:
#|hide
#|default_exp core

# core

> Fast scripts using daemon mode

In [2]:
#|export
import argparse,socket,struct,sys

In [3]:
import time
from io import BytesIO
from socketserver import TCPServer,StreamRequestHandler
from fastcore.parallel import threaded

In [4]:
#|export
def _send_string(w, s):
    if isinstance(s,str): s = s.encode('utf-8')
    l = len(s)
    w(struct.pack(f'!L{l}s', l, s))

In [5]:
#|export
def send_strings(w, ss):
    "Send a list of variable length utf-8-encoded strings"
    for s in ss: _send_string(w, s)

In [6]:
ss = ['from fastcore.all import *\n', 'nbprocess_clean --stdin']
f = BytesIO()
send_strings(f.write, ss)
b = f.getvalue(); b

b'\x00\x00\x00\x1bfrom fastcore.all import *\n\x00\x00\x00\x17nbprocess_clean --stdin'

In [7]:
#|export
def readlen(r):
    "Read the length of the next string"
    return struct.unpack('!L', r(4))[0]

In [8]:
readlen(BytesIO(b).read)

27

In [9]:
#|export
def recv_strings(r):
    "Receive a 2-tuple of variable length utf-8-encoded strings"
    res = []
    for _ in range(2):
        l = readlen(r)
        s = struct.unpack(f'{l}s', r(l))[0].decode('utf-8')
        res.append(s)
    return res

In [10]:
recv_strings(BytesIO(b).read)

['from fastcore.all import *\n', 'nbprocess_clean --stdin']

In [11]:
#|export
def _socket_det(port,host,dgram):
    # Source: https://github.com/fastai/fastcore/blob/da9ca219c86190d22f4dbf2c3cad619c477d64a4/fastcore/net.py#L222-L225
    if isinstance(port,int): family,addr = socket.AF_INET,(host or socket.gethostname(),port)
    else: family,addr = socket.AF_UNIX,port
    return family,addr,(socket.SOCK_STREAM,socket.SOCK_DGRAM)[dgram]

In [12]:
#|export
def start_client(port, host=None, dgram=False):
    "Create a `socket` client on `port`, with optional `host`, of type `dgram`"
    # Source: https://github.com/fastai/fastcore/blob/da9ca219c86190d22f4dbf2c3cad619c477d64a4/fastcore/net.py#L242-L247
    family,addr,typ = _socket_det(port,host,dgram)
    s = socket.socket(family, typ)
    s.connect(addr)
    return s

In [13]:
class TestHandler(StreamRequestHandler):
    def handle(self): self.wfile.write(self.rfile.readline())

In [14]:
addr = host,port = 'localhost',9999
@threaded
def _f():
    with TCPServer(addr,TestHandler) as srv: srv.handle_request()
_f()
time.sleep(0.2) # wait for server to start

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/seem/.pyenv/versions/3.9.7/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/seem/.pyenv/versions/3.9.7/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/ft/0gnvc3ts5jz4ddqtttp6tjvm0000gn/T/ipykernel_63649/253354604.py", line 4, in _f
  File "/Users/seem/.pyenv/versions/3.9.7/lib/python3.9/socketserver.py", line 452, in __init__
    self.server_bind()
  File "/Users/seem/.pyenv/versions/3.9.7/lib/python3.9/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 48] Address already in use


In [15]:
with start_client(port, host) as client:
    with client.makefile('wb') as f: f.write('hello world\n'.encode())
    with client.makefile('rb') as f: print(f.read().decode(), end='')

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
#|export
def send_recv(streams, port, host=None, dgram=False):
    "Wraps `start_client`, `send_strings`, and `recv_strings`"
    with start_client(port, host=host, dgram=dgram) as client:
        with client.makefile('wb') as f: send_strings(f.write, streams)
        with client.makefile('rb') as f: return recv_strings(f.read)

In [ ]:
#|export
def _fastdaemon_client(port, host, args):
    args = ' '.join(args)
    stdin = sys.stdin.read() if not sys.stdin.isatty() else ''
    stdout,stderr = send_recv((stdin,args), port, host)
    sys.stderr.write(stderr)
    sys.stdout.write(stdout)

In [ ]:
#|export
def fastdaemon_client(argv=None):
    "Forward `sys.args` and `sys.stdin` to `fastdaemon_server` and write response `stdout` and `stderr`"
    if argv is None: argv = sys.argv[1:]
    p = argparse.ArgumentParser(description=fastdaemon_client.__doc__)
    p.add_argument('port', type=int, help='Port to connect to')
    p.add_argument('--host', type=str, help='Host to connect to', default=None)
    args,rest = p.parse_known_args(argv)
    args.args = rest
    _fastdaemon_client(**vars(args))

## Export -

In [16]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()